# Notebook 4: LangChain basics

# LangChain
LangChain is a framework for developing applications powered by large language models (LLMs).

<img src="https://drive.google.com/uc?id=11muIjdnCaZksAwoaFAZjL8hAH5Z0mnmb" />

### Step 1: Use GPT via LangChain

* We use the AzureChatOpenAI API
* Working with azure is security-wise approved

In [1]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_deployment="gpt-35-turbo",
    api_version="2023-06-01-preview",
    temperature=0.7,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [2]:
res = llm.invoke('Hello, how do I exit vim?')  # Send an API request to Azure OpenAI via LangChain. Simple!
print(res.content)  # No need for complicated parsing. Sweet.

To exit vim, you need to follow these steps:

1. Press the Esc key to ensure you are in command mode.
2. Type `:q` to quit vim. If you have unsaved changes, vim will not exit and will display an error message.
3. If you have made changes and want to discard them, you can type `:q!` to forcefully quit vim without saving.
4. If you have made changes and want to save them before quitting, type `:wq` to write the changes to the file and exit vim.

Remember, vim can be a bit complex for beginners. If you find yourself struggling, you can refer to vim's documentation or use other text editors that are more user-friendly.


### Step 2: Use a ChatPromptTemplate to create your prompt

We can define prompt templates.
* These templates consist of System, Human and AI messages.
* The template contains multiple variables that will be given different values.
* The PromptTemplate is like Python's f-strings (and in fact uses it)

In [3]:
from langchain.prompts import ChatPromptTemplate

In [4]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("assistant", "I'm doing well, thanks!"),  # Poor AI, we put words in his mouth!
    ("human", "{user_input}"),
])

# We use the Template to create an actual prompt by giving the required arguments
messages = template.format_messages(
    name="Check Bot",
    user_input="What is your name?"
)

response = llm(messages)
print(response.content)

/tmp/ipykernel_8414/3037245385.py:14: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  response = llm(messages)


My name is Check Bot. How can I assist you today?


### Step 3: Use Few-shot

* When asking an LLM to do something for the first time, without any examples, we call it **zero shot**
* Supplying the LLM with an example for what we want can drastically increase its performance
* If a single example is given to the LLM, we call it **1-shot**
* If multiple examples are supplied, we call it **few shot**
* https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/few_shot_examples_chat

This technique is based on the [Language Models are Few-Shot Learners paper](https://arxiv.org/abs/2005.14165)

In [5]:
from langchain.prompts import FewShotChatMessagePromptTemplate

In [6]:
# Use a list of dictionaries to store examples. Each example is a dictionary.
# Notice that each example has a user input and an expected result.

examples = [
    {"input": "2 + 2", "output": "The amazing answer is 4!"},
    {"input": "2 + 3", "output": "The unexpected answer is 5!"},
]

In [7]:
# A single Template will be used for all examples

example_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("assistant", "{output}"),
    ]
)

In [8]:
# We supply the FewShotChatMessagePromptTemplate with a Template and a dictionary

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt_template,  # This is how the examples should look like
    examples=examples,  # And this is the actual examples
)

In [9]:
# Template for the final prompt
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wonderous wizard of math."),
        few_shot_prompt,  # This encapsulates all the examples
        ("human", "{input}"),
    ]
)

In [10]:
final_prompt = prompt_template.format_messages(
    input="6 + 3?"
)

final_prompt  # Let's see our prompt

[SystemMessage(content='You are a wonderous wizard of math.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The amazing answer is 4!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='2 + 3', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The unexpected answer is 5!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='6 + 3?', additional_kwargs={}, response_metadata={})]

In [11]:
# First let's see what GPT responds without examples
print(llm("3 + 6").content)

The sum of 3 and 6 is 9.


In [12]:
response = llm(final_prompt)
print(response.content)

The magical answer is 9!


## Example selectors
* LangChain allows you to pick examples in dynamic ways.
* You can select examples by length, relevance, similarity and more
* https://python.langchain.com/docs/modules/model_io/prompts/example_selectors/


## Exercises
* Use few shots and a system prompt to create a chatbot as follows -
* Input is a name of a country
* Output should be in the format of `Capital | population | gdp`

## Summary
* We can use LangChain to communicate with OpenAI with greater ease
* We can use `ChatPromptTemplate` as 'recipes' for generating prompts
* When an LLM tries to solve a problem it has never seen before we can it **zero shot**
* When an LLM solves a problem with a single/multiple examples, we call it **one shot**/**few shot**
* We can use `FewShotChatMessagePromptTemplate` to create few shot prompts

## Resources

* https://python.langchain.com/docs/get_started/introduction.html  # docs
* https://github.com/langchain-ai/langchain  # github
* docs: https://api.python.langchain.com/en/latest/chat_models/langchain.chat_models.azure_openai.AzureChatOpenAI.html